In [2]:
from dotenv import load_dotenv
import os
from crewai import Crew, Task
from crewai.tools import tool
from crewai import Agent
import yfinance as yf

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"


class Tools:

    @tool("One month stock price history")
    def stock_price(ticker: str) -> str:
        """
        Useful to get a month's worth of stock price data as CSV.
        The input of this tool should a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()


class Agents:

    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            # 주가를 가져오는 툴
            tools=[
                Tools.stock_price,
            ],
        )


class Tasks:

    def technical_analysis(self, agent):
        return Task(
            description="Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.",
            expected_output=f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions.",
            # context="Stock analysis for {company}. Use historical price data and tools to provide entry points and price targets.",
            agent=agent,
            output_file="technical_analysis.md",
        )


from crewai.process import Process
from langchain_openai import ChatOpenAI

agents = Agents()
tasks = Tasks()

technical_analyst = agents.technical_analyst()

technical_task = tasks.technical_analysis(technical_analyst)

crew = Crew(
    agents=[
        technical_analyst,
    ],
    tasks=[
        technical_task,
    ],
    verbose=True,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=True,
)

result = crew.kickoff(
    inputs=dict(
        company="Advanced Micro Devices",
    ),
)

Overriding of current TracerProvider is not allowed


# Agent: Crew Manager
## Task: Conduct a detailed technical analysis of the price movements of Advanced Micro Devices's stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.


# Agent: Crew Manager
## Thought: I need to gather historical stock price data for Advanced Micro Devices (AMD) to conduct a detailed technical analysis. This data will help identify key support and resistance levels, chart patterns, and other technical indicators.
## Using tool: One month stock price history
## Tool Input: 
"{\"ticker\": \"AMD\"}"
## Tool Output: 
Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
2025-01-06 00:00:00-05:00,129.07000732421875,130.72999572753906,127.36000061035156,129.5500030517578,48020200,0.0,0.0
2025-01-07 00:00:00-05:00,130.50999450683594,131.7100067